In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import re
from functools import reduce


- "https://www.bankofengland.co.uk/-/media/boe/files/markets/asset-purchase-facility/gilt-purchase-operational-results.xlsx"
- "https://www.bankofengland.co.uk/-/media/boe/files/markets/asset-purchase-facility/gilt-sales-time-series.xlsx"

In [ ]:
df_buy = pd.ExcelFile("downloads/gilt-purchase-operational-results.XLSX")

In [ ]:
df_buy.sheet_names

In [ ]:
df_buy = pd.read_excel("downloads/gilt-purchase-operational-results.XLSX", sheet_name="APF Gilts", header=1)

In [ ]:
df_buy.columns

In [ ]:
df_boe = df_buy[['Operation date','Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)']].copy()

In [ ]:
df_boe['Operation date'] = pd.to_datetime(df_boe['Operation date'], format='%d-%m-%Y')
df_boe.groupby('Operation date').sum().cumsum().plot()

In [ ]:
df_boe.set_index('Operation date', inplace=True)

In [ ]:
df_boe.head()

In [ ]:
df_boe.cumsum(axis=0).plot()

## Accrual Calculations

In [ ]:
df_buy.head()

In [ ]:
from bgs.load_gilt_details import load_csv_blocks
from bgs.load_bgs_prices import load_prices
from bgs.load_bgs_amounts import load_bgs_amounts
from bgs.bgs_utils import clean_date, clean_percentage
from bgs.gilt_analytics import coupon_schedule


In [ ]:
details = load_csv_blocks("downloads/BGSDetails.csv")

conv_details = details['Conventionals']
conv_details['%'] = conv_details['%'].apply(clean_percentage)
conv_details['Sequence'] = conv_details['Sequence'].apply(lambda x: np.int64(x))

new_details = details['Index-Linked New-style']
new_details['%'] = new_details['%'].apply(clean_percentage)
new_details['Sequence'] = new_details['Sequence'].apply(lambda x: np.int64(x))

old_details = details['Index-Linked Old-style']
old_details['%'] = old_details['%'].apply(clean_percentage)
old_details['Sequence'] = old_details['Sequence'].apply(lambda x: np.int64(x))

date_variables = ["Latest redemption date","Issue date","First coupon payable on date",]
for date_var in date_variables:
    conv_details[date_var] = conv_details[date_var].apply(clean_date)
    old_details[date_var] = old_details[date_var].apply(clean_date)
    new_details[date_var] = new_details[date_var].apply(clean_date)

In [ ]:
conv_details.columns

In [ ]:
conv_details.head()

In [ ]:
conv_details.replace(to_replace=32112, value=32120, inplace=True)

In [ ]:
conv_details[conv_details['Sequence']== 32120]

In [ ]:
df_sell = pd.ExcelFile("downloads/gilt-sales-time-series.XLSX")

In [ ]:
df_sell.sheet_names

In [ ]:
df_sell = pd.read_excel("downloads/gilt-sales-time-series.XLSX", sheet_name="APF gilt sales", header=1)

In [ ]:
df_sell.columns

In [ ]:
df_sell['Operation date'] = pd.to_datetime(df_sell['Operation date'], format='%d-%m-%Y')
last_date = df_sell['Operation date'].min()

There are three dates - the date when the buying stopped, the maturity date of the bond, and the date when the selling starts

In [ ]:
# first bond in the list
gilt_isin = "GB0031829509"

purchase_history = df_buy[df_buy['ISIN'] == gilt_isin][['Operation date','Settlement date', 'Total allocation (proceeds £mn)','Total allocation (nominal £mn)']].copy()
start_purchase = purchase_history['Operation date'].min().strftime("%Y-%m-%d")
bond = conv_details[conv_details['ISIN Code']==gilt_isin].copy()
issue_date=bond['Issue date'].dt.strftime("%Y-%m-%d").values[0]
maturity_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
first_cpn_date=bond['First coupon payable on date'].dt.strftime("%Y-%m-%d").values[0]
last_cpn_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
coupon=bond['%'].values[0]/100

In [ ]:
start_purchase

In [ ]:
schedule = coupon_schedule(start_purchase, issue_date, maturity_date, first_cpn_date, last_cpn_date)

In [ ]:
coupons_due = [pd.to_datetime([x.serialNumber()-1], unit="D", origin=pd.Timestamp("1899-12-31")) for x in schedule]

In [ ]:

# coupons_due = [x for x in coupons_due if x < last_date]


In [ ]:
bond_payments = pd.DataFrame({
    'coupon': [coupon]*len(coupons_due),
    'principal': [0]*(len(coupons_due)-1)+[1]
}, index=reduce(lambda x,y: x.union(y),coupons_due ))


In [ ]:
ownership = purchase_history.groupby('Settlement date')[['Total allocation (proceeds £mn)','Total allocation (nominal £mn)']].sum().cumsum().resample('D').last().ffill()

In [ ]:
# first date
ownership.index.max()


In [ ]:
# filter coupons due before last date
cash_flows_due = bond_payments[bond_payments.index < last_date].copy()

In [ ]:
cash_flows_due = cash_flows_due.merge(ownership, left_index=True, right_index=True, how='left').ffill()
cash_flows_due['cash_flow'] = (cash_flows_due['coupon']+ cash_flows_due['principal']) * cash_flows_due['Total allocation (nominal £mn)'] 

In [ ]:
cash_flows_due

In [ ]:
cash_flows_due['cash_flow'].sum()

In [ ]:
df_buy['ISIN'].unique()

In [ ]:
payments = []
for gilt_isin in df_buy['ISIN'].unique():
    purchase_history = df_buy[df_buy['ISIN'] == gilt_isin][['Operation date','Settlement date', 'Total allocation (proceeds £mn)','Total allocation (nominal £mn)']].copy()
    start_purchase = purchase_history['Operation date'].min().strftime("%Y-%m-%d")
    bond = conv_details[conv_details['ISIN Code']==gilt_isin].copy()
    issue_date=bond['Issue date'].dt.strftime("%Y-%m-%d").values[0]
    maturity_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
    first_cpn_date=bond['First coupon payable on date'].dt.strftime("%Y-%m-%d").values[0]
    last_cpn_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
    coupon=bond['%'].values[0]/100

    schedule = coupon_schedule(start_purchase, issue_date, maturity_date, first_cpn_date, last_cpn_date)
    coupons_due = [pd.to_datetime([x.serialNumber()-1], unit="D", origin=pd.Timestamp("1899-12-31")) for x in schedule]
    bond_payments = pd.DataFrame({
        'coupon': [coupon]*len(coupons_due),
        'principal': [0]*(len(coupons_due)-1)+[1]
    }, index=reduce(lambda x,y: x.union(y),coupons_due ))

    ownership = purchase_history.groupby('Settlement date')[['Total allocation (proceeds £mn)','Total allocation (nominal £mn)']].sum().cumsum().resample('D').last().ffill()
    cash_flows_due = bond_payments[bond_payments.index < last_date].copy()
    cash_flows_due = cash_flows_due.merge(ownership, left_index=True, right_index=True, how='left').ffill()
    cash_flows_due[gilt_isin] = (cash_flows_due['coupon']+ cash_flows_due['principal']) * cash_flows_due['Total allocation (nominal £mn)'] 
    payments.append(cash_flows_due[[gilt_isin]])



In [ ]:
boe_cashflows_by_isin = pd.concat(payments, axis=1).fillna(0)

In [ ]:
boe_cashflows_by_isin.head()

In [ ]:
boe_cf = boe_cashflows_by_isin.sum(axis=1).cumsum().reset_index().set_index('index').rename(columns={
    0: 'Cumulative Cash Flows (£mn)'
})
boe_exposure = df_boe.groupby(
    'Operation date'
    )['Total allocation (proceeds £mn)'].sum().cumsum(axis=0).resample('D').last().ffill().reset_index().set_index('Operation date')

In [ ]:
boe_exposure

In [ ]:
boe_exposure = boe_exposure.merge(
        boe_cf, left_index=True, right_index=True, how='left').ffill().fillna(0)



In [ ]:
boe_exposure

In [ ]:
(boe_exposure['Total allocation (proceeds £mn)'] - boe_exposure['Cumulative Cash Flows (£mn)']).plot()

In [ ]:
payments_to_hmt = pd.read_csv('downloads/MT6A.csv',skiprows=8,names=["Date","MT6A"])
q_start =  payments_to_hmt.loc[payments_to_hmt['Date']=="2009 Q1"].index[0]
q_end = payments_to_hmt.loc[payments_to_hmt['Date']=="1993 JAN"].index[0] -1
payments_to_hmt_q = payments_to_hmt.iloc[q_start:q_end].copy()
payments_to_hmt_q.tail()
payments_to_hmt_q['date']=payments_to_hmt_q['Date'].apply(lambda x: f"{x.split(' Q')[0]}-{int(x.split(' Q')[1])*3}-01")
payments_to_hmt_q['date']=pd.to_datetime(payments_to_hmt_q['date'], format='%Y-%m-%d')
payments_to_hmt_q.set_index('date', inplace=True)
payments_to_hmt_q.head()

In [ ]:
boe_exposure = boe_exposure.merge(
        payments_to_hmt_q.cumsum().resample('D').last().ffill().reset_index().set_index('date'), left_index=True, right_index=True, how='left').ffill().fillna(0)



In [ ]:
boe_exposure

In [ ]:
(boe_exposure['Total allocation (proceeds £mn)'] - boe_exposure['Cumulative Cash Flows (£mn)'] + boe_exposure['MT6A']).plot()

### Base Rate Funding

In [ ]:
boe_base_rate = pd.read_csv("downloads/Bank Rate history and data  Bank of England Database.csv")
boe_base_rate.head()
boe_base_rate['Date Changed'] = pd.to_datetime(boe_base_rate['Date Changed'], format="%d %b %y", errors='coerce')
base_rate = boe_base_rate.copy().set_index('Date Changed').resample('D').ffill().reset_index().set_index('Date Changed')
base_rate.index = pd.to_datetime(base_rate.index)
base_rate.head()

In [ ]:
base_rate.index

In [ ]:
boe_exposure.index = pd.to_datetime(boe_exposure.index, format="%Y-%m-%d %H:%M:%S")

In [ ]:
boe_exposure.index

In [ ]:
boe_exposure = boe_exposure.merge(
        base_rate, left_index=True, right_index=True, how='left')

In [ ]:
boe_exposure.head()

In [ ]:
funding_cost = (boe_exposure['Total allocation (proceeds £mn)'] * (boe_exposure['Rate'] / 100) / 365).cumsum().reset_index().set_index('Operation date')

In [ ]:
funding_cost.plot()

In [ ]:
funding_cost.head()

In [ ]:
boe_exposure = boe_exposure.merge(
        funding_cost.rename(columns={
                0:'funding_cost'
        }), left_index=True, right_index=True, how='left')

In [ ]:
boe_exposure.head()

In [ ]:
(boe_exposure['Total allocation (proceeds £mn)'] - boe_exposure['Cumulative Cash Flows (£mn)'] + boe_exposure['MT6A'] - boe_exposure['funding_cost']).plot()

### Gain / Loss on a position

### Positions when selling starts

In [ ]:
df_end_qe=df_buy[['Operation date','Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)','ISIN']].copy()
holdings_by_isin=df_end_qe.groupby('ISIN')[['Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)']].sum()
holdings_by_isin.rename(columns={
    'Total allocation (proceeds £mn)': 'cost_of_investment',
    'Total allocation (nominal £mn)': 'investment'
}, inplace=True)
holdings_by_isin

In [ ]:
df_qt=df_sell[['Operation date','Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)','ISIN']].copy()
sales_by_isin=df_qt.groupby('ISIN')[['Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)']].sum()
sales_by_isin.rename(columns={
    'Total allocation (proceeds £mn)': 'sales_proceeds',
    'Total allocation (nominal £mn)': 'investment_reduction'
}, inplace=True)
sales_by_isin

In [ ]:
positions = holdings_by_isin.merge(sales_by_isin, left_index=True, right_index=True, how='left').fillna(0)
positions.head()

In [ ]:
def p_l(row):
    if row['investment_reduction'] > 0:
        return row['sales_proceeds'] - (row['cost_of_investment'] / row['investment']) * row['investment_reduction']
    else:
        return 0
positions['p_l'] = positions.apply(p_l, axis=1)
positions.head()

In [ ]:
positions['p_l'].sum()

In [ ]:
conv_details.columns

In [ ]:
df_boe_exit = df_sell[['Operation date','Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)']].copy()
df_boe_exit['Operation date'] = pd.to_datetime(df_boe_exit['Operation date'], format='%d-%m-%Y')
df_boe_exit.set_index('Operation date', inplace=True)
df_boe_exit.cumsum(axis=0).plot()

In [ ]:
bonds = df_buy[['ISIN','Bond\n']].drop_duplicates()

In [ ]:
from collections import defaultdict
isin_bond_map = defaultdict(list)
for row in bonds.itertuples():
    isin_bond_map[row.ISIN].append(row._2)
for isin, bond in isin_bond_map.items():
    if len(bond) > 1:
        raise Exception(f"There are duplicates in the BoE list: {isin}")
isin_bond_map['GB0008881541']

In [ ]:
gilt_level_buys = df_buy[[
    'ISIN',
    'Operation date',
    'Total allocation (proceeds £mn)',
    'Total allocation (nominal £mn)'
    ]].copy()
gilt_level_buys.set_index('Operation date', inplace=True)
gilt_level_buys.index = pd.to_datetime(gilt_level_buys.index, format='%d-%m-%Y').to_period('M').to_timestamp('M')
gilt_level_buys.reset_index(inplace=True)


In [ ]:
gilt_buys_ts = gilt_level_buys.pivot_table(
    index='Operation date', 
    columns='ISIN', 
    values='Total allocation (nominal £mn)', aggfunc='sum')

In [ ]:
gilt_buys_ts.columns

In [ ]:
gilt_buys_ts = gilt_buys_ts.fillna(0).cumsum()
gilt_buys_ts.head()
# TODO: Clip once the Bond has matured

In [ ]:
gilt_buys_total = gilt_level_buys.reset_index().groupby([ 'ISIN']).agg(
    Total_Allocation_Proceeds=('Total allocation (proceeds £mn)', 'sum'),
    Total_Allocation_Nominal=('Total allocation (nominal £mn)', 'sum'),
    Last_Purchase_Month=('Operation date', 'max')
)
gilt_buys_total.reset_index(inplace=True)
gilt_buys_total.head()
    

In [ ]:
isin_bond_details = conv_details[['ISIN Code', 'Latest redemption date','%','Sequence']].copy()

In [ ]:
[x for x in gilt_buys_total['ISIN'] if x not in isin_bond_details['ISIN Code'].to_list()]

In [ ]:
gilt_buys_total['ISIN'][0]

In [ ]:
isin_bond_details[isin_bond_details['ISIN Code']=='GB0002404191']['Latest redemption date'].values[0]

In [ ]:
gilt_buys_total['maturity_date'] = gilt_buys_total['ISIN'].map(
    lambda x: isin_bond_details[
        isin_bond_details['ISIN Code']==x
        ]['Latest redemption date'].values[0]
    )
gilt_buys_total['bgs_id'] = gilt_buys_total['ISIN'].map(
    lambda x: isin_bond_details[
        isin_bond_details['ISIN Code']==x
        ]['Sequence'].values[0]
    )

In [ ]:
gilt_buys_total.head()

In [ ]:
gilt_buys_total['maturity_date'] = pd.to_datetime(
    gilt_buys_total['maturity_date'], format="%d %b %Y"
)

In [ ]:
bond_sales = df_sell[['ISIN','Bond\n']].drop_duplicates()
bond_sales

In [ ]:
isin_bond_sales_map = defaultdict(list)
for row in bond_sales.itertuples():
    isin_bond_sales_map[row.ISIN].append(row._2)
for isin, bond in isin_bond_sales_map.items():
    if len(bond) > 1:
        raise Exception(f"There are duplicates in the BoE list: {isin}")
isin_bond_sales_map['GB0008881541']

In [ ]:
isin_bond_sales_map

In [ ]:
gilt_level_sales = df_sell[[
    'ISIN',
    'Operation date',
    'Total allocation (proceeds £mn)',
    'Total allocation (nominal £mn)'
    ]].copy()
gilt_level_sales.set_index('Operation date', inplace=True)
gilt_level_sales.index = pd.to_datetime(gilt_level_sales.index, format='%d-%m-%Y').to_period('M').to_timestamp('M')
gilt_level_sales.reset_index(inplace=True)
gilt_level_sales.head()

In [ ]:
sales_by_isin=gilt_level_sales.groupby('ISIN')[['Total allocation (nominal £mn)', 'Total allocation (proceeds £mn)']].sum().reset_index()

In [ ]:
sales_by_isin['maturity_date'] = sales_by_isin['ISIN'].map(
    lambda x: isin_bond_details[
        isin_bond_details['ISIN Code']==x
        ]['Latest redemption date'].values[0]
    )
sales_by_isin['maturity_date'] = pd.to_datetime(
    sales_by_isin['maturity_date'], format="%d %b %Y"
)

In [ ]:
matured_sales = sales_by_isin['maturity_date'] > pd.Timestamp('2025-04-30')
len(matured_sales)- matured_sales.sum()

In [ ]:
matured = gilt_buys_total['maturity_date'] > pd.Timestamp.now()

In [ ]:
sales_by_isin[matured_sales].shape, gilt_buys_total[matured].shape

In [ ]:
check_sales = sales_by_isin[matured_sales]['Total allocation (nominal £mn)'].sum()
check_sales

In [ ]:
unsold = [x for x in gilt_buys_total[matured]['ISIN'].to_list() if x not in sales_by_isin[matured_sales]['ISIN'].to_list()]  
unsold

In [ ]:
# conv_details[conv_details['ISIN Code']==unsold[0]].T

In [ ]:
boe_portfolio = gilt_buys_total[matured].set_index('ISIN').merge(
    sales_by_isin[matured_sales][[
        'ISIN',
        'Total allocation (nominal £mn)',
        'Total allocation (proceeds £mn)'
        ]].set_index('ISIN'),
        how='left',
        left_index=True,
        right_index=True,
        suffixes=('_buy', '_sell')
        ).copy().fillna(0).reset_index()

In [ ]:
boe_portfolio.head()

In [ ]:
# boe_portfolio[boe_portfolio['ISIN']==unsold[0]]

In [ ]:
assert boe_portfolio['Total allocation (nominal £mn)'].sum() == check_sales

In [ ]:
boe_portfolio['current_position'] = boe_portfolio['Total_Allocation_Nominal'] - boe_portfolio['Total allocation (nominal £mn)']
boe_portfolio['position_cost'] = boe_portfolio['Total_Allocation_Proceeds'] - boe_portfolio['Total allocation (proceeds £mn)']

In [ ]:
boe_portfolio['Coupon']=boe_portfolio['ISIN'].map(
    lambda x: isin_bond_details[
        isin_bond_details['ISIN Code']==x
        ]['%'].values[0]
    )

In [ ]:
boe_portfolio.groupby('Coupon')['current_position'].sum().plot(
    kind='bar', figsize=(12, 6), title='BoE Portfolio by Coupon'
)

In [ ]:
boe_portfolio.groupby('Coupon')['position_cost'].sum().cumsum()

In [ ]:
price_df=load_prices("downloads/BGSPrices.csv")

In [ ]:
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')

In [ ]:
price_df.loc['Apr 2025',32100]

In [ ]:
boe_portfolio['mkt_price'] = boe_portfolio['bgs_id'].map(
    lambda x: price_df.loc['Apr 2025', int(x)].values[0])

In [ ]:
boe_portfolio.head()

In [ ]:
boe_portfolio['p&l'] = boe_portfolio['mkt_price'] * boe_portfolio['current_position']/100 - boe_portfolio['position_cost']  

In [ ]:
boe_portfolio['p&l'].sum()

In [ ]:
boe_portfolio.groupby('Coupon')['p&l'].sum().plot(
    kind='bar', figsize=(12, 6), title='BoE Portfolio Loses by Coupon'
)

In [ ]:
boe_portfolio.groupby('Coupon')['p&l'].sum().cumsum()

In [ ]:

tables = load_bgs_amounts("downloads/BGSAmounts.csv")

In [ ]:
conv = tables['Conventionals']
conv.replace("Redeemed", "", inplace=True)
conv = conv.apply(pd.to_numeric, errors='coerce')
conv.fillna(0, inplace=True)
conv.index = pd.Index([clean_date(x) for x in list(conv.index)]).to_period('M').to_timestamp('M')
conv.columns = [np.int64(x.strip()) for x in conv.columns]

In [ ]:
conv.loc['30 Apr 2025',32100]

In [ ]:
boe_portfolio['tot_outstanding'] = boe_portfolio['bgs_id'].map(
    lambda x: conv.loc['30 Apr 2025', x])

In [ ]:
conv.loc['30 Apr 2025',32100]

In [ ]:
boe_portfolio['pct_ownership']=boe_portfolio['current_position']/boe_portfolio['tot_outstanding'].astype(float).fillna(0)

In [ ]:
boe_portfolio['pct_ownership'].plot(kind='hist')

In [ ]:
boe_portfolio.plot(x='pct_ownership',y='p&l', kind='scatter')

In [ ]:
boe_portfolio[boe_portfolio['pct_ownership']>0.5][['ISIN','p&l', 'pct_ownership']].sort_values(by='p&l', ascending=True)

In [ ]:
conv_details.loc[conv_details['ISIN Code']=='GB00BFWFPP71'].T

In [ ]:
boe_end_purchase_port =  gilt_buys_total.loc[gilt_buys_total['maturity_date'] > gilt_buys_total['Last_Purchase_Month'].max()]

In [ ]:
boe_end_purchase_port['year'] = boe_end_purchase_port['maturity_date'].dt.year
boe_end_purchase_port.groupby('year')['Total_Allocation_Nominal'].sum().plot(kind='bar', x='year', y='Total_Allocation_Nominal')

In [ ]:
boe_end_purchase_port.groupby('year')['Total_Allocation_Nominal'].sum().cumsum().plot(kind='bar', x='year', y='Total_Allocation_Nominal')

In [ ]:
conv_details[conv_details['Sequence']==32100]

In [ ]:
boe_end_purchase_port.head()

In [ ]:
boe_portfolio